Используем другую модель (например, GradientBoosting)

Создадим больше признаков прямо в эксперименте

Используем другие гиперпараметры

In [1]:
"""
ML-эксперимент 2: GradientBoosting с расширенным набором признаков
Используется вторая версия датасета (предобработанные данные)
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import joblib
from clearml import Task, Dataset

# === 1. Инициализация Task в ClearML ===
task = Task.init(
    project_name='CustomerPropensityTracking',
    task_name='ML_Experiment_2_GradientBoosting',
    task_type=Task.TaskTypes.training
)

# === 2. Загрузка данных и создание расширенных признаков ===
logger = task.get_logger()
logger.report_text("Загрузка данных и создание расширенных признаков...")

# Загружаем данные
df = pd.read_csv('/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')

# Преобразуем дату
df['tr_date'] = pd.to_datetime(df['tr_date'], dayfirst=True)
df['year_month'] = df['tr_date'].dt.to_period('M')

# Целевая категория
df['is_garden'] = (df['item_group'] == 'Оборудование для сада и дачи').astype(int)
df = df.sort_values(['client', 'tr_date'])

# Группировка по клиенту и месяцу
client_monthly = df.groupby(['client', 'year_month']).agg(
    total_amount=('amount', 'sum'),
    purchase_count=('bcode', 'nunique'),
    garden_purchase=('is_garden', 'max'),
    unique_items=('item', 'nunique'),
    avg_transaction=('amount', 'mean')
).reset_index()

# Целевая переменная
client_monthly['target'] = client_monthly.groupby('client')['garden_purchase'].shift(-1)
client_monthly = client_monthly.dropna(subset=['target'])
client_monthly['target'] = client_monthly['target'].astype(int)

# Создание дополнительных признаков
client_monthly = client_monthly.sort_values(['client', 'year_month'])

# Лаги
for lag in [1, 2, 3]:
    client_monthly[f'amount_lag_{lag}'] = client_monthly.groupby('client')['total_amount'].shift(lag)
    client_monthly[f'count_lag_{lag}'] = client_monthly.groupby('client')['purchase_count'].shift(lag)
    client_monthly[f'garden_lag_{lag}'] = client_monthly.groupby('client')['garden_purchase'].shift(lag)

# Скользящие статистики
client_monthly['amount_rolling_mean_3'] = client_monthly.groupby('client')['total_amount'].transform(
    lambda x: x.rolling(3, min_periods=1).mean()
)
client_monthly['garden_rolling_sum_3'] = client_monthly.groupby('client')['garden_purchase'].transform(
    lambda x: x.rolling(3, min_periods=1).sum()
)

# Время с последней покупки в категории
client_monthly['months_since_garden'] = client_monthly.groupby('client')['garden_purchase'].apply(
    lambda x: (x == 1).cumsum().shift().fillna(0)
).reset_index(level=0, drop=True)

# Удаляем NaN
client_monthly = client_monthly.dropna()

# Признаки
feature_cols = [
    'total_amount', 'purchase_count', 'garden_purchase', 'unique_items', 'avg_transaction',
    'amount_lag_1', 'count_lag_1', 'garden_lag_1',
    'amount_lag_2', 'count_lag_2', 'garden_lag_2',
    'amount_rolling_mean_3', 'garden_rolling_sum_3', 'months_since_garden'
]

X = client_monthly[feature_cols]
y = client_monthly['target']

# Разделение по времени (последний месяц - тест)
client_monthly['year_month_str'] = client_monthly['year_month'].astype(str)
last_month = client_monthly['year_month_str'].max()

train_val = client_monthly[client_monthly['year_month_str'] != last_month]
test = client_monthly[client_monthly['year_month_str'] == last_month]

X_train_val = train_val[feature_cols]
y_train_val = train_val['target']
X_test = test[feature_cols]
y_test = test['target']

# Дополнительное разделение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val
)

logger.report_text(f"Признаки: {len(feature_cols)}")
logger.report_text(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# === 3. Логирование гиперпараметров ===
hyperparams = {
    'n_estimators': 150,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_samples_split': 10,
    'min_samples_leaf': 4,
    'subsample': 0.8,
    'random_state': 42
}

task.connect(hyperparams)
logger.report_text(f"Гиперпараметры: GradientBoostingClassifier, {hyperparams}")

# === 4. Обучение модели ===
logger.report_text("Обучение GradientBoosting...")
model = GradientBoostingClassifier(**hyperparams)
model.fit(X_train, y_train)

# === 5. Логирование метрик ===
# Валидация
y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]

val_metrics = {
    'accuracy': accuracy_score(y_val, y_val_pred),
    'f1': f1_score(y_val, y_val_pred),
    'roc_auc': roc_auc_score(y_val, y_val_proba)
}

for metric_name, metric_value in val_metrics.items():
    logger.report_scalar(title='Validation Metrics', series=metric_name, value=metric_value, iteration=0)
    task.get_logger().report_single_value(name=f'val_{metric_name}', value=metric_value)

logger.report_text(f"Метрики на валидации: {val_metrics}")

# Динамика обучения (loss по итерациям)
for i, loss in enumerate(model.train_score_):
    logger.report_scalar(title='Training Loss', series='GradientBoosting', value=loss, iteration=i)

# === 6. Сохранение модели ===
model_path = 'gradientboosting_model_v1.pkl'
joblib.dump(model, model_path)
task.upload_artifact('model', model_path)
logger.report_text(f"Модель сохранена как артефакт: {model_path}")

# === 7. Тестирование ===
y_test_pred = model.predict(X_test)
y_test_proba = model.predict_proba(X_test)[:, 1]

test_metrics = {
    'accuracy': accuracy_score(y_test, y_test_pred),
    'f1': f1_score(y_test, y_test_pred),
    'roc_auc': roc_auc_score(y_test, y_test_proba)
}

for metric_name, metric_value in test_metrics.items():
    logger.report_scalar(title='Test Metrics', series=metric_name, value=metric_value, iteration=0)
    task.get_logger().report_single_value(name=f'test_{metric_name}', value=metric_value)

logger.report_text(f"Метрики на тесте: {test_metrics}")

# === 8. Завершение ===
task.close()
print("ML Experiment 2 завершен!")
print(f"Task ID: {task.task_id}")
print(f"ROC-AUC на валидации: {val_metrics['roc_auc']:.4f}")
print(f"ROC-AUC на тесте: {test_metrics['roc_auc']:.4f}")
print(f"Использовано признаков: {len(feature_cols)}")

ClearML Task: created new task id=a5f54a5f25ea4f44bec2b2ecf9999433
ClearML results page: https://app.clear.ml/projects/79b0194396f14ec69257352bc2de6e92/experiments/a5f54a5f25ea4f44bec2b2ecf9999433/output/log
Загрузка данных и создание расширенных признаков...
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
Признаки: 14
Train: (36896, 14), Val: (9224, 14), Test: (2825, 14)
Гиперпараметры: GradientBoostingClassifier, {'n_estimators': 150, 'learning_rate': 0.1, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 4, 'subsample': 0.8, 'random_state': 42}
Обучение GradientBoosting...
Метрики на валидации: {'accuracy': 0.7283174327840416, 'f1': np.float64(0.47726324572382145), 'roc_auc': np.float64(0.7359660348536377)}
Модель сохранена как артефакт: gradientboosting_model_v1.pkl
Метрики на тесте: {'accuracy': 0.7713274336283186, 'f1': np.float64(0.4773462783171521), 'roc_auc': np.float64(0.7649068907563025)}
ML Experiment 2 завершен!
Task ID: 

Task ID: a5f54a5f25ea4f44bec2b2ecf9999433

ROC-AUC на валидации: 0.7360

ROC-AUC на тесте: 0.7649

Признаков: 14